# Spectral Mesh Flattening

by Ana + Gabrielle

In [468]:
import trimesh
import numpy as np

In [502]:
def boundary(mesh):
    
    """ A function to find the boundary edges and vertices of a mesh. 
        Inputs:
        mesh: a trimesh mesh. 
        Outputs: 
        boundary_vertices: a list of the indices of vertices on the boundary. 
        next_vertex: a dictionary to find the next vertex on the boundary. 
    """

    next_vertex = {}
    boundary_vertices = []

    edges = mesh.edges
    vertex_faces = mesh.vertex_faces
    
    for i in range(len(edges)):

        edge = edges[i]
        v1, v2 = edge
        faces = [j for j in vertex_faces[v1] if j != -1 and j in vertex_faces[v2]]
        
        # Boundary edges are edges that are only in one face
        if len(faces) == 1:

            next_vertex[v1] = v2
            boundary_vertices = boundary_vertices + [v1, v2]
    
    # Remove any duplicates
    boundary_vertices = np.unique(np.array(boundary_vertices))
    
    return boundary_vertices, next_vertex

In [496]:
## This is to export the boundaries coloured. 
# face = trimesh.load("Face.obj")
# b_verts, next_vertex = boundary(face)
# face.visual.vertex_colors[b_verts] = [255, 0, 0, 1]
# face.visual.vertex_colors[1] = [0, 255, 0, 1]
# face.visual.vertex_colors[3] = [0, 0, 255, 1]
# face.export("boundaries.obj")

'# https://github.com/mikedh/trimesh\nv -6.52889010 5.49005320 -6.22479870 0.40000000 0.40000000 0.40000000\nv -5.35236360 -5.50825500 -8.62843320 0.00000000 1.00000000 0.00000000\nv -5.81779670 -5.07507990 -8.55370240 0.40000000 0.40000000 0.40000000\nv -5.96906280 -4.76623150 -9.31085210 0.00000000 0.00000000 1.00000000\nv 6.86571880 4.88439180 -7.15710880 0.40000000 0.40000000 0.40000000\nv 6.53557400 5.43919660 -6.24136880 0.40000000 0.40000000 0.40000000\nv 5.35236360 -5.50825500 -8.62843320 1.00000000 0.00000000 0.00000000\nv 5.96906280 -4.76623150 -9.31085210 1.00000000 0.00000000 0.00000000\nv 5.82194710 -5.07033540 -8.55861280 0.40000000 0.40000000 0.40000000\nv -3.76459310 -6.91110420 -5.99737550 0.40000000 0.40000000 0.40000000\nv -3.90613940 -6.93728640 -6.53265000 1.00000000 0.00000000 0.00000000\nv -4.43992230 -6.46998600 -7.07713320 1.00000000 0.00000000 0.00000000\nv -4.36145310 -6.44226360 -6.47173600 0.40000000 0.40000000 0.40000000\nv 3.90613940 -6.93728640 -6.532650

In [503]:
def circle_boundary(mesh, centre = [0, 0]):

    """ A function to map the boundary of a mesh to a circle centered at the origin. 
        Inputs:
        mesh: a trimesh mesh. 
        centre: optional argument to change the centre of the circle. 
        
        Outputs: 
        new_boundary_values: an array of boundary values (2 dimensional) on the circle. 
    """

    vertices = mesh.vertices

    range_x = (np.max(vertices[:, 0]) - np.min(vertices[:, 0]))/2
    range_y = (np.max(vertices[:, 1]) - np.min(vertices[:, 1]))/2
    range_z = (np.max(vertices[:, 2]) - np.min(vertices[:, 2]))/2

    # Keep the circle roughly the same size as the mesh by choosing half the maximum range
    # as the radius 
    radius = np.max([range_x, range_y, range_z])

    # find the boundary information, and intialize the output 
    b_verts, next_vertex = boundary(mesh)
    output = np.zeros((len(vertices), 2))
    weights = {}
    
    # Calculate the "weights" of each edge and store in a dictionary 
    # to be used to determine how far from the other vertices each edge 
    for i in range(len(b_verts)):
        weights[b_verts[i]] = np.linalg.norm(vertices[b_verts[i]] - vertices[next_vertex[b_verts[i]]])
    total = sum(weights.values(), 0.0)
    weights = {k: v / total for k, v in weights.items()}

    # Start with one point set at 0 degrees, and then increase the angle each time (since arc length is
    # proportional to angle)
    angle_sum = 0
    v1 = b_verts[0]
    output[v1] = [centre[0] + radius * np.cos(angle_sum), centre[1] + radius * np.sin(angle_sum)]

    for i in range(len(weights)):
        
        angle = weights[v1] * 2 * np.pi
        angle_sum += angle

        # calculate the new positions
        new_x = centre[0] + radius * np.cos(angle_sum)
        new_y = centre[1] + radius * np.sin(angle_sum)

        v1 = next_vertex[v1]
        output[v1] = [new_x, new_y]

    # make sure the outputs are in the same order as the b_verts 
    new_boundary_values = output[b_verts]

    return new_boundary_values

In [504]:
## This cell exports the mesh with the boundary vertices transformed to the 2d circle. 
# import matplotlib.pyplot as plt
# %matplotlib inline

# face = trimesh.load("Face.obj")
# xy = np.array(circle_boundary(face, centre = [0, 0]))
# x = xy[:, 0]
# y = xy[:, 1]
# plt.scatter(x, y, marker = 'x')
# b_verts, _ = boundary(face)
# face.vertices[b_verts] = np.hstack((xy, -12*np.ones((len(x), 1))))
# face.export("CircleExperiment.obj")